<a href="https://colab.research.google.com/github/GuDing0820/Colab/blob/main/%E4%BA%BA%E8%87%89%E8%AE%8A%E8%AE%8A%E8%AE%8A_%E6%9C%80%E7%B5%82%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install omegaconf
!pip install gradio
!pip install diffusers
!pip install transformers
!pip install safetensors
!pip install huggingface_hub
!pip install accelerate
!pip install torch
!pip install controlnet_aux
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=99449ba0ec2aa400662a9e73f60ac1ed52107d6dff190cbffe0a5b1734f5e701
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 11.9 

In [ ]:
!sudo apt-get update
!sudo apt-get install git-lfs


!git lfs install


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [44.8 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,131 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,009 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,400 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Hit:13 https://ppa.l

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import gradio as gr
import torch
from torch import autocast
from tqdm.auto import tqdm
import numpy as np
import requests
import PIL
import queue
from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionControlNetPipeline, ControlNetModel,UniPCMultistepScheduler
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image

def generate_image(input_image, style, prompt, negative_prompt): # -> np.ndarray:

    controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose", torch_dtype=torch.float16)
    openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
    openpose_image = input_image
    openpose_image = openpose(openpose_image)
    #device = "cuda"
    #model_id = "Xinxiang0820/junji_ito"
    style_to_model_id = {
    "伊藤潤二": "Xinxiang0820/junji_ito",
    "吉卜力": "Xinxiang0820/Ghi-bli-style",
    "辛普森": "Xinxiang0820/the_simpsons",
    "迪士尼": "Xinxiang0820/Disney_style",
    "火影忍者": "Xinxiang0820/Naruto",
}
    model_id = style_to_model_id[style]
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        model_id,
        controlnet = controlnet,
        torch_dtype=torch.float16,
        safety_checker=None,
        use_safetensors=True
        #use_auth_token=True

    )

       #pipe = pipe.to(device)
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    #pipe.enable_xformers_memory_efficient_attention()
    pipe.enable_model_cpu_offload()
    # 將 PIL 圖像轉換為 PyTorch 張量
    #init_image = input_image.convert("RGB") #torch.from_numpy(input_image).permute(2, 0, 1).float().to(device) / 255.0
    #init_image = init_image.unsqueeze(0)

    #generator = torch.Generator(device='cuda').manual_seed(1024)
    generator = torch.manual_seed(0)
    #with autocast('cuda'):
    output_image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=openpose_image,
            #strength=0.5,
            #guidance_scale=7.5,
            generator=generator,
            num_inference_steps=20
        ).images[0]

    # 將 PyTorch 張量轉換回 NumPy 數組
    #output_image = (output_image.squeeze(0).permute(1, 2, 0).cpu().numpy() * 255.0).clip(0, 255).astype(np.uint8)

    return output_image

    input_image = gr.Image(type="pil", label="上傳圖片")
    style_input = gr.Dropdown(choices=["伊藤潤二", "吉卜力", "辛普森", "迪士尼", "火影忍者"], label="選擇風格")  # Add dropdown for style selection
    prompt_input = gr.Textbox(lines=1, label="提示")
    negative_prompt_input = gr.Textbox(lines=1, label="負面提示")

# 定義 Gradio 輸入和輸出界面組件
style_input = gr.Dropdown(choices=["伊藤潤二", "吉卜力", "辛普森", "迪士尼", "火影忍者"], label="選擇風格")  # Add dropdown for style selection
#input_image = gr.inputs.Image(type="numpy", label="上傳圖片")
input_image = gr.Image(type="pil", label="上傳圖片")
#output_image = gr.outputs.Image(type="numpy",label="轉換後的圖片")
output_image = gr.Image(type="pil",label="轉換後的圖片")
prompt_input = gr.Textbox(lines=1, label="提示")
negative_prompt_input = gr.Textbox(lines=1, label="負面提示")

# 創建 Gradio 介面並啟動
gr.Interface(
    fn=generate_image,
    inputs=[input_image, style_input, prompt_input, negative_prompt_input],
    outputs=output_image,
    title="人臉變變變",
    description="上傳一張圖片，並輸入提示和負面提示，將圖片轉換為新風格。",
    allow_flagging="auto"

).launch(share=True,debug=True,enable_queue=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

<ipython-input-4-e23137651140>:80: GradioDeprecationWarning: The `enable_queue` parameter has been deprecated. Please use the `.queue()` method instead.
  gr.Interface(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a9bae2227b61954c52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/20 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a9bae2227b61954c52.gradio.live
